<a href="https://colab.research.google.com/github/Ajinkyadata/Coursera-sequence-models/blob/master/add_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import glob
import pickle
import numpy as np
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential
from keras.layers import *
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, LambdaCallback, EarlyStopping

from termcolor import colored

Using TensorFlow backend.


In [0]:
chars = '0123456789+'

In [0]:
num=len(chars)

In [0]:
char_ind = dict((c,i) for i, c in enumerate(chars))
ind_char=dict((i,c) for i,c in enumerate(chars))

In [0]:
def generate():
  one=np.random.randint(0,100)
  two=np.random.randint(0,100)

  ans=str(one)+str('+')+str(two)
  label=str(one+two)

  return ans, label

In [6]:
generate()

('42+34', '76')

In [0]:
hidden=128
Tx=5

In [52]:
model =Sequential([
                   SimpleRNN(hidden, input_shape=(None,num)),
                   RepeatVector(Tx),
                   SimpleRNN(hidden, return_sequences=True),
                   TimeDistributed(Dense(num ,activation='softmax'))
])

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy'], 
              )

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_7 (SimpleRNN)     (None, 128)               17920     
_________________________________________________________________
repeat_vector_4 (RepeatVecto (None, 5, 128)            0         
_________________________________________________________________
simple_rnn_8 (SimpleRNN)     (None, 5, 128)            32896     
_________________________________________________________________
time_distributed_4 (TimeDist (None, 5, 11)             1419      
Total params: 52,235
Trainable params: 52,235
Non-trainable params: 0
_________________________________________________________________


In [0]:
#vectorisation

def vec(eg, label):
  x=np.zeros((Tx, num))
  y=np.zeros((Tx, num))

  diff_x= Tx-len(eg)
  diff_y=Tx-len(label)

  for i,c in enumerate(eg):
    x[i+diff_x, char_ind[c]]=1
  for i in range(diff_x):
    x[i,char_ind['0']]=1

  for i,c in enumerate(label):
    y[i+diff_y, char_ind[c]]=1
  for i in range(diff_y):
    y[i,char_ind['0']]=1

    return x,y


In [10]:
egg, lbl= vec('3+43','56')
print(egg)
print(lbl)

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]
[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]]


In [0]:
#devec
def devec(eg):
  result= [ind_char[np.argmax(vec)] for i, vec in enumerate(eg)]
  return ''.join(result)

In [12]:
devec(egg)

'03+43'

In [0]:
#create dataset
def create_data(num_eg=4000):
  x=np.zeros((num_eg, Tx, num))
  y=np.zeros((num_eg, Tx, num))

  for i in range(num_eg):
    q, a = generate()

    x[i],y[i] = vec(q,a)

  return x,y;

In [49]:
xx, yy= create_data()

print(xx.shape)

(4000, 5, 11)


In [50]:
print(devec(xx[9]))

70+42


In [62]:
#training

call_b= LambdaCallback(
    on_epoch_end=lambda e, l: print(e, end='--')
    )

early= EarlyStopping(monitor='val_loss', patience=20)
model.fit(xx,yy,epochs=500, batch_size=256 ,validation_split=0.2,
          verbose=1, callbacks=[early,call_b])

"""scores = model.evaluate(xx, yy,verbose=0)
print ("Train loss : %.3f" % scores[0])
print ("Train accuracy : %.3f" % scores[1])
print ("Training finished") """

Train on 3200 samples, validate on 800 samples
Epoch 1/500
3200/3200 [==============================] - 0s 89us/step - loss: 0.0016 - accuracy: 0.7649 - val_loss: 0.0699 - val_accuracy: 0.7387
0--Epoch 2/500
3200/3200 [==============================] - 0s 87us/step - loss: 0.0016 - accuracy: 0.7651 - val_loss: 0.0703 - val_accuracy: 0.7395
1--Epoch 3/500
3200/3200 [==============================] - 0s 87us/step - loss: 0.0016 - accuracy: 0.7649 - val_loss: 0.0691 - val_accuracy: 0.7385
2--Epoch 4/500
3200/3200 [==============================] - 0s 91us/step - loss: 0.0016 - accuracy: 0.7646 - val_loss: 0.0688 - val_accuracy: 0.7398
3--Epoch 5/500
3200/3200 [==============================] - 0s 85us/step - loss: 0.0015 - accuracy: 0.7649 - val_loss: 0.0689 - val_accuracy: 0.7400
4--Epoch 6/500
3200/3200 [==============================] - 0s 85us/step - loss: 0.0015 - accuracy: 0.7651 - val_loss: 0.0696 - val_accuracy: 0.7398
5--Epoch 7/500
3200/3200 [==============================] - 0s

'scores = model.evaluate(xx, yy,verbose=0)\nprint ("Train loss : %.3f" % scores[0])\nprint ("Train accuracy : %.3f" % scores[1])\nprint ("Training finished") '

In [75]:
x_test, y_test= create_data(10)

pred=model.predict(x_test)
for i,hat in enumerate(pred):
  y=devec(y_test[i])
  y_hat=devec(hat)
  col='green'
  if y!=y_hat:
    col='red'
  out='input: '+devec(x_test[i])+' our prediction: '+y_hat+' actual output:'+y
  print(colored(out, col))

input: 09+43 our prediction: 01152 actual output:00052
input: 91+75 our prediction: 00166 actual output:00166
input: 14+13 our prediction: 00127 actual output:00027
input: 85+47 our prediction: 00132 actual output:00132
input: 61+43 our prediction: 01104 actual output:00104
input: 56+53 our prediction: 01109 actual output:00109
input: 89+80 our prediction: 00169 actual output:00169
input: 053+0 our prediction: 06162 actual output:00053
input: 77+70 our prediction: 01147 actual output:00147
input: 78+99 our prediction: 01177 actual output:00177
